In [0]:
silver = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.silver_transform"

df_trips = spark.read.table(f"{silver}.trips")
df_payments = spark.read.table(f"{silver}.payments")


In [0]:
silver = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.silver_transform"

df_trips = spark.read.table(f"{silver}.trips")
df_payments = spark.read.table(f"{silver}.payments")
display(df_trips)
display(df_payments)



trip_id,driver_id,customer_id,vehicle_id,trip_start_time,trip_end_time,start_location,end_location,distance_km,fare_amount,payment_method,trip_status,last_updated_timestamp
1,29,185,29,2025-07-31T18:00:11Z,2025-07-31T18:07:11Z,New Victoriahaven,New Cynthiamouth,9.09,28.41,Card,Cancelled,2025-09-06T01:55:37Z
2,2,180,2,2025-08-29T04:00:11Z,2025-08-29T04:36:11Z,West Wanda,Donaldburgh,22.68,76.3,Card,Ongoing,2025-08-25T21:45:53Z
3,43,182,43,2025-08-25T09:00:11Z,2025-08-25T09:36:11Z,North Robert,West Donald,29.23,47.0,Cash,Completed,2025-09-07T12:33:38Z
4,4,152,4,2025-08-10T21:00:11Z,2025-08-10T21:16:11Z,Chadville,West Marcusmouth,25.07,35.69,Wallet,Completed,2025-09-04T10:41:23Z
5,23,132,23,2025-09-12T15:00:11Z,2025-09-12T15:37:11Z,Schneiderview,Ochoaton,33.2,49.46,Card,Ongoing,2025-09-20T04:33:10Z
6,31,80,31,2025-07-11T06:00:11Z,2025-07-11T06:59:11Z,Williamsbury,Monroemouth,7.46,21.01,Wallet,Completed,2025-08-23T06:21:23Z
7,15,172,15,2025-07-12T06:00:11Z,2025-07-12T06:31:11Z,New Dana,Carpenterport,36.28,128.96,Cash,Completed,2025-09-05T17:04:37Z
8,4,12,4,2025-07-27T07:00:11Z,2025-07-27T07:14:11Z,Whitneybury,West Kennethmouth,33.88,57.18,Wallet,Ongoing,2025-08-22T14:35:00Z
9,27,160,27,2025-07-03T16:00:11Z,2025-07-03T16:05:11Z,West Christopherhaven,Nicholasberg,16.03,51.53,Wallet,Ongoing,2025-09-15T09:56:31Z
10,41,126,41,2025-09-05T05:00:11Z,2025-09-05T05:20:11Z,Ashleyborough,North Morganburgh,20.53,42.27,Card,Cancelled,2025-08-22T12:08:26Z


payment_id,trip_id,customer_id,payment_method,payment_status,amount,transaction_time,last_updated_timestamp,online_payment_status,process_timestamp
1,274,126,Cash,Success,38.15,2025-09-17T13:00:12Z,2025-08-30T13:40:53Z,offline,2025-11-21T02:21:24.321998Z
10,133,70,Cash,Failed,69.99,2025-07-31T13:00:12Z,2025-09-11T10:30:16Z,offline,2025-11-21T02:21:24.321998Z
100,265,92,Wallet,Success,90.93,2025-07-15T13:00:12Z,2025-09-01T15:06:16Z,offline,2025-11-21T02:21:24.321998Z
1000,903,43,Card,Pending,96.61,2025-08-29T13:00:12Z,2025-08-30T01:59:42Z,offline,2025-11-21T02:21:24.321998Z
101,761,107,Cash,Success,8.75,2025-08-10T13:00:12Z,2025-09-01T13:16:51Z,offline,2025-11-21T02:21:24.321998Z
102,139,134,Wallet,Pending,59.42,2025-07-04T13:00:12Z,2025-08-25T19:45:46Z,offline,2025-11-21T02:21:24.321998Z
103,604,80,Card,Success,45.54,2025-09-17T13:00:12Z,2025-09-13T02:33:29Z,offline,2025-11-21T02:21:24.321998Z
104,434,122,Cash,Failed,73.37,2025-07-04T13:00:12Z,2025-09-04T19:07:43Z,offline,2025-11-21T02:21:24.321998Z
105,168,80,Cash,Pending,68.93,2025-08-30T13:00:12Z,2025-08-30T18:12:56Z,offline,2025-11-21T02:21:24.321998Z
106,814,87,Card,Failed,29.59,2025-08-02T13:00:12Z,2025-09-19T08:05:12Z,offline,2025-11-21T02:21:24.321998Z


In [0]:
silver = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.silver_transform"

df_trips = spark.read.table(f"{silver}.trips")
df_payments = spark.read.table(f"{silver}.payments")
display(df_trips)
display(df_payments)


trip_id,driver_id,customer_id,vehicle_id,trip_start_time,trip_end_time,start_location,end_location,distance_km,fare_amount,payment_method,trip_status,last_updated_timestamp
1,29,185,29,2025-07-31T18:00:11Z,2025-07-31T18:07:11Z,New Victoriahaven,New Cynthiamouth,9.09,28.41,Card,Cancelled,2025-09-06T01:55:37Z
2,2,180,2,2025-08-29T04:00:11Z,2025-08-29T04:36:11Z,West Wanda,Donaldburgh,22.68,76.3,Card,Ongoing,2025-08-25T21:45:53Z
3,43,182,43,2025-08-25T09:00:11Z,2025-08-25T09:36:11Z,North Robert,West Donald,29.23,47.0,Cash,Completed,2025-09-07T12:33:38Z
4,4,152,4,2025-08-10T21:00:11Z,2025-08-10T21:16:11Z,Chadville,West Marcusmouth,25.07,35.69,Wallet,Completed,2025-09-04T10:41:23Z
5,23,132,23,2025-09-12T15:00:11Z,2025-09-12T15:37:11Z,Schneiderview,Ochoaton,33.2,49.46,Card,Ongoing,2025-09-20T04:33:10Z
6,31,80,31,2025-07-11T06:00:11Z,2025-07-11T06:59:11Z,Williamsbury,Monroemouth,7.46,21.01,Wallet,Completed,2025-08-23T06:21:23Z
7,15,172,15,2025-07-12T06:00:11Z,2025-07-12T06:31:11Z,New Dana,Carpenterport,36.28,128.96,Cash,Completed,2025-09-05T17:04:37Z
8,4,12,4,2025-07-27T07:00:11Z,2025-07-27T07:14:11Z,Whitneybury,West Kennethmouth,33.88,57.18,Wallet,Ongoing,2025-08-22T14:35:00Z
9,27,160,27,2025-07-03T16:00:11Z,2025-07-03T16:05:11Z,West Christopherhaven,Nicholasberg,16.03,51.53,Wallet,Ongoing,2025-09-15T09:56:31Z
10,41,126,41,2025-09-05T05:00:11Z,2025-09-05T05:20:11Z,Ashleyborough,North Morganburgh,20.53,42.27,Card,Cancelled,2025-08-22T12:08:26Z


payment_id,trip_id,customer_id,payment_method,payment_status,amount,transaction_time,last_updated_timestamp,online_payment_status,process_timestamp
1,274,126,Cash,Success,38.15,2025-09-17T13:00:12Z,2025-08-30T13:40:53Z,offline,2025-11-21T02:21:24.321998Z
10,133,70,Cash,Failed,69.99,2025-07-31T13:00:12Z,2025-09-11T10:30:16Z,offline,2025-11-21T02:21:24.321998Z
100,265,92,Wallet,Success,90.93,2025-07-15T13:00:12Z,2025-09-01T15:06:16Z,offline,2025-11-21T02:21:24.321998Z
1000,903,43,Card,Pending,96.61,2025-08-29T13:00:12Z,2025-08-30T01:59:42Z,offline,2025-11-21T02:21:24.321998Z
101,761,107,Cash,Success,8.75,2025-08-10T13:00:12Z,2025-09-01T13:16:51Z,offline,2025-11-21T02:21:24.321998Z
102,139,134,Wallet,Pending,59.42,2025-07-04T13:00:12Z,2025-08-25T19:45:46Z,offline,2025-11-21T02:21:24.321998Z
103,604,80,Card,Success,45.54,2025-09-17T13:00:12Z,2025-09-13T02:33:29Z,offline,2025-11-21T02:21:24.321998Z
104,434,122,Cash,Failed,73.37,2025-07-04T13:00:12Z,2025-09-04T19:07:43Z,offline,2025-11-21T02:21:24.321998Z
105,168,80,Cash,Pending,68.93,2025-08-30T13:00:12Z,2025-08-30T18:12:56Z,offline,2025-11-21T02:21:24.321998Z
106,814,87,Card,Failed,29.59,2025-08-02T13:00:12Z,2025-09-19T08:05:12Z,offline,2025-11-21T02:21:24.321998Z


In [0]:
from pyspark.sql.functions import to_date
df_joined = (
    df_trips
    .join(df_payments, "trip_id", "inner")
    .drop(df_trips.customer_id)     # remove the ambiguous column
)



In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

df_daily_revenue = (
    df_joined
    .groupBy("trip_id")
    .agg(
        sum("amount").alias("total_revenue"),
        countDistinct("trip_id").alias("total_trips"),
        countDistinct("customer_id").alias("unique_customers"),
        avg("amount").alias("avg_revenue_per_trip")
    )
)
display(df_daily_revenue)


trip_id,total_revenue,total_trips,unique_customers,avg_revenue_per_trip
463,251.63,1,3,83.87666666666667
471,87.68,1,1,87.68
392,61.09,1,1,61.09
897,47.78,1,1,47.78
243,85.51,1,1,85.51
737,93.07,1,1,93.07
623,48.56,1,1,48.56
540,81.13,1,1,81.13
516,52.9,1,2,26.45
31,107.53999999999999,1,2,53.769999999999996


In [0]:
from pyspark.sql.functions import to_date

df_joined = df_joined.withColumn("trip_date", to_date("trip_start_time"))


In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

df_daily_revenue = (
    df_joined
    .groupBy("trip_date")
    .agg(
        sum("amount").alias("total_revenue"),
        countDistinct("trip_id").alias("total_trips"),
        countDistinct("customer_id").alias("unique_customers"),
        avg("amount").alias("avg_revenue_per_trip")
    )
)
display(df_daily_revenue)


trip_date,total_revenue,total_trips,unique_customers,avg_revenue_per_trip
2025-06-29,585.1600000000001,6,9,58.516000000000005
2025-08-14,588.3,8,10,58.83
2025-08-16,698.37,8,11,63.48818181818182
2025-07-29,1419.5299999999997,14,21,64.5240909090909
2025-08-31,476.0400000000001,5,8,52.893333333333345
2025-07-18,505.71000000000004,6,10,50.571000000000005
2025-09-04,654.23,8,11,54.51916666666667
2025-09-10,401.81000000000006,4,6,57.40142857142858
2025-08-13,870.23,12,17,48.346111111111114
2025-09-11,1025.2999999999997,12,17,53.96315789473683


In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS 
thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.gold_transform
""")


DataFrame[]

In [0]:
gold = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.gold_transform"

df_daily_revenue.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{gold}.daily_revenue_summary")
